# Задачи RMQ/RSQ

**RMQ (range minimum/maximum query)** - задача о нахождении минимума/максимума на данном отрезке

**RSQ (range sum query)** - похожая задача, в которой нужно найти сумму чисел на заданном отрезке

## Префиксные суммы

Дан некоторый массив чисел `a=[1, -5, 3, 2, 4, -6, 1, -4, 4, 2, -4]` и нужно уметь отвечать на запросы вида "сумма чисел на отрезке от `l` до `r`". 

Конечно, можно каждый раз насчитывать эту сумму, но если таких запросов приходит, например, `10^5` штук, а чисел в массиве будет примерно столько же, то работать это все будет неприлично много — $O(n)$

Массив префиксных сумм `p` на `i`-й позиции содержит сумму чисел на отрезке `[0...i]` массива `a`

In [27]:
def pref_sums(a):
    n = len(a)
    p = [0] * n
    for i in range(1, n):
        p[i] = p[i-1] + a[i]
    return p


Добавим в начало массива `a` фиктивный 0, чтобы было удобнее работать с индексацией

In [31]:
a = [0, 1, -5, 3, 2, 4, -6, 1, -4, 4, 2, -4]
p = pref_sums(a)
print(p)

[0, 1, -4, -1, 1, 5, -1, 0, -4, 0, 2, -2]


Теперь для ответа на запрос вида `summ(left, right)` нужно взять префиксную сумму на отрезке `[0...r]` и вычесть из нее сумму на отрезке `[0...left]`

In [ ]:
def summ(left, right, p):
    return p[right] - p[left - 1]

**Сложность.** Построение массива `p` займет $O(n)$, а ответ на запрос $O(1)$, так как это просто операция вычитания.

## Префиксные суммы на матрице

Пусть у нас есть матрица $m\in\text{Mat}_{n\times n}(\mathbb{R})$. Для работы уже имеющегося кода добавим нулевой столбец и строку, которые будут заполнены фиктивными нулями. Например, 
\begin{equation*}
    m=\begin{pmatrix}
        0&0&0&0&0\\
        0&1&-1&-3&4\\
        0&-1&2&2&-1\\
        0&-1&-2&1&3\\
        0&4&-2&3&-1
    \end{pmatrix}
\end{equation*}

Мы получаем запросы вида "найти сумму чисел в прямоугольнике". Прямоугольник в данном случае - это какая-то подматрица

Тогда, мы можем создать матрицу префиксных сумм `p`, в которой позиция `p[i][j]` хранит сумму от `m[0][0]` до `m[i][j]`. Давайте заполним такую матрицу:
\begin{equation*}
    p=\begin{pmatrix}
        0&0&0&0&0\\
        0&1&0&-3&1\\
        0&0&1&0&3\\
        0&-1&-2&-2&4\\
        0&3&0&3&8
    \end{pmatrix}
\end{equation*}

In [35]:
def pref_summ_2d(m):
    n = len(m)
    p = [[0] * n for _ in range(n)]
    for i in range(1, n):
        for j in range(1, n):
            p[i][j] = p[i-1][j] + p[i][j-1] - p[i-1][j-1] + m[i][j]
    return p

m = [[0, 0, 0, 0, 0], [0, 1, -1, -3, 4], [0, -1, 2, 2, -1], [0, -1, -2, 1, 3], [0, 4, -2, 3, -1]]
pref_summ_2d(m)

[[0, 0, 0, 0, 0],
 [0, 1, 0, -3, 1],
 [0, 0, 1, 0, 3],
 [0, -1, -2, -2, 4],
 [0, 3, 0, 3, 8]]

Тогда, чтобы получить сумму в подматрице, где строки от $i_1$ до $i_2$, а столбцы от $j_1$ до $j_2$, равна 
$$
\text{sum}(i_1, j_1, i_2, j_2) = p[i_2][j_2] - p[i_1-1][j_2] - p[i_2][j_1-1] + p[i_1-1][j_1-1]
$$
То есть береи сумму большого прямоугольника $(1,1) \to (i_2,j_2)$, вычитаем:
- верхнюю полоску над прямоугольником,
- левую полоску слева,

а угол $(i_1-1, j_1-1)$ вычли дважды, поэтому его нужно добавить обратно.

In [37]:
def summ_2d(i1, j1, i2, j2, p):
    return p[i2][j2] - p[i1-1][j2] - p[i2][j1-1] + p[i1-1][j1-1]

In [38]:
summ_2d(2, 2, 3, 3, pref_summ_2d(m))

3

Префиксные суммы плохи тем, что если в исходном массиве меняется какой-то элемент, то построенный массив префсумм становится неактуален и нужно строить новый

## sqrt-декомпозиция

Пусть имеется массив
$$a=[3, 5, -1, 4, -3, -3, 2, 1, 6, 9, 3, 1, 2, 5, 1, 1]$$
длины $n$ и мы снова хотим отвечать на вопросы о сумме/минимуме/максимуме на отрезке

Давайте равномерно разобьем этот массив на $\sqrt{n}$ блоков, то есть в каждом блоке одинаковое количество элементов. Тогда, в каждом блоке будет $\sqrt{n}$ элементов

В случае с массивом `a` мы получим 4 блока по 4 элемента в каждом
$$a=\left[\boxed{3, 5, -1, 4}, \boxed{-3, -3, 2, 1}, \boxed{6, 9, 3, 1}, \boxed{2, 5, 1, 1}\right]$$
Давайте создадим массив `block` длиной $sqrt{n}=4$. Тогда, каждая ячейка этого массива будет "обслуживать" элементы `a` с индексами
$$\boxed{0...3}\boxed{4...7}\boxed{8...11}\boxed{12...15}$$
Теперь в каждую ячейку мы запишем минимум на соответствующем отрезке, для нашего массива получится
$$\text{block}=[-1, -3, 1, 1]$$

Допустим, пришел запрос "найти минимум на отрезке от `l=6` до `r=13`". Тогда, нам нужно посмотреть на элементы `a[6]`-`a[7]` и `a[12]`=`a[13]`, а минимум на отрезке от 8 до 11 мы уже знаем - это `block[2]=1`

Аналогичные рассуждения будут и для нахождения суммы на отрезке, напишем код для случая RSQ

In [43]:
a = [3, 5, -1, 4, -3, -3, 2, 1, 6, 9, 3, 1, 2, 5, 1, 1]
size_block = 4
block = [0] * (len(a) // size_block + 1)

for i in range(len(a)):
    block[i // size_block] += a[i]

def sum_arr(left: int, right: int) -> int:
    s = 0
    while left <= right:
        # если left и right в разных блоках и left стоит в начале блока
        if left // size_block != right // size_block and left % size_block == 0:
            s += a[left]
            left += 1
        else:
            s += a[left]
            left += 1
    return s


print(sum_arr(2, 9), sum(a[2:10])) 

15 15


Суммы одинаковые, а значит `sum_arr()` работает корректно, а главное быстрее на больших данных

Если же в `a` меняется какой-то элемент, то нужно будет пересчитать один конкретный блок, который обслуживает этот элемент

**Сложность.** Пройти от `l` до правой границы блока (в который попадает `l`) займет $\sqrt{n}$; пройти от `r` до левой границы блока займет $\sqrt{n}$; целых блоков между `l` и `r` будет не более $\sqrt{n}$ штук. В итоге — $O(\sqrt{n})$ 

## Максимумы на подотрезках

Нужно реализовать структуру данных для эффективного вычисления максимумов подряд идущих элементов массива

### Дано
- Одно натуральное число $N\ (1 \leqslant N \leqslant 10^5)$ — количество чисел в массиве
- Массив из $N$ чисел от $1$ до $10^5$ — элементы массива.
- Одно натуральное число $К\ (1 \leqslant K \leqslant 3\cdot 10^4)$ — количество запросов на вычисление максимума.
- Массив из $K$ пар вида `(l;r)` — номера левого и правого элементов отрезка массива (считается, что элементы массива нумеруются с единицы)

In [ ]:
n = 10
arr = [18, 16, 20, 2, 53, 57, 39, 97, 21, 59]
size_block = 250
inf = 10 ** 6

Размер блока можно выбрать любым. Если блоки оказываются слишком большими, ускорения не будет и запросы будут обрабатываться линейно.

Для достижения оптимальной асимптотики размер блока лучше взять $\sqrt{n}$

In [57]:
block = [-inf] * (len(a) // size_block + 1)

for i in range(len(arr)):
    block[i // size_block] = max(block[i // size_block], arr[i])

def max_arr(left: int, right: int) -> int:
    ans = -inf
    while left <= right:
        # если left и right в разных блоках и left стоит в начале блока
        if left // size_block != right // size_block and left % size_block == 0:
            ans = max(block[left // size_block], ans)
            left += size_block
        else:
            ans = max(arr[left], ans)
            left += 1
    return ans

k = 6
reqs = [(3, 10), (1, 10), (3, 4), (9, 10), (8, 10), (5, 8)]
for i in range(k):
    left, right = reqs[i][0], reqs[i][1]
    print(max(arr[left-1:right]) == max_arr(left - 1, right - 1))   # проверим, верно ли работает наша функция


True
True
True
True
True
True


Мы не получили `False`, так что функция работает корректно (да, тестов мало - но поверьте на слово)